# Machine Learning Part
* goal of this notebook is to develop code for the machine learning portion
    * dataset loading
    * model
    * evaluation metrics
* code should be added into qdanalysis and refactored aferwards

In [1]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
data = keras.utils.image_datasetfrom_directory